In [1]:
!pip install transformers datasets

from datasets import load_dataset

data_conclusion_abstract = load_dataset("json", data_files="limitations_2024/limitations_training_bart_1_2024.json")
data_conclusion_abstract = data_conclusion_abstract["train"].train_test_split(test_size=0.1)
data_tokenized_papers = load_dataset("json", data_files="limitations_2024/limitations_training_bart_2_2024.json")
data_tokenized_papers = data_tokenized_papers["train"].train_test_split(test_size=0.1)
data_full_papers = load_dataset("json", data_files="limitations_2024/limitations_training_bart_3_2024.json")
data_full_papers = data_full_papers["train"].train_test_split(test_size=0.1)

In [2]:
from transformers import AutoTokenizer

model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_input = 512
max_output = 256

def tokenize_data(text):
    model_input = tokenizer(text["input"], max_length=max_input, truncation=True, padding=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(text["target"], max_length=max_output, truncation=True, padding=True)

    model_input["labels"] = labels["input_ids"]
    return model_input

In [3]:
tokenized_conclusion_abstract = data_conclusion_abstract.map(tokenize_data, batched=True)
tokenized_tokenized_papers = data_tokenized_papers.map(tokenize_data, batched=True)
tokenized_full_papers = data_full_papers.map(tokenize_data, batched=True)

Map:   0%|          | 0/758 [00:00<?, ? examples/s]

/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/85 [00:00<?, ? examples/s]

Map:   0%|          | 0/758 [00:00<?, ? examples/s]

Map:   0%|          | 0/85 [00:00<?, ? examples/s]

Map:   0%|          | 0/758 [00:00<?, ? examples/s]

Map:   0%|          | 0/85 [00:00<?, ? examples/s]

In [4]:
!pip install -U transformers
!pip install 'accelerate>=0.26.0
!pip install transformers[torch]
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments

def train_model(dataset, model_name, output_folder):
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    training_args = Seq2SeqTrainingArguments(
        output_dir=output_folder,
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        weight_decay=0.01,
        save_total_limit=2,
        num_train_epochs=3,
        predict_with_generate=True,
        logging_dir=f"{output_folder}/logs",
        logging_steps=10,
        eval_strategy = "epoch",
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=tokenizer,
    )
    print(tokenizer.pad_token, tokenizer.pad_token_id)
    print(tokenizer.eos_token, tokenizer.eos_token_id)
    
    trainer.train()
    trainer.save_model(f"{output_folder}/final")
    tokenizer.save_pretrained(f"{output_folder}/final")
    return trainer
    

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/bin/bash: -c: line 1: unexpected EOF while looking for matching `''
/bin/bash: -c: line 2: syntax error: unexpected end of file


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
!pip install 'accelerate>=0.26.0
! pip install -U transformers

trainer_abstract = train_model(tokenized_conclusion_abstract, model_name, "./model_output_abstract")
trainer_tokenized = train_model(tokenized_tokenized_papers, model_name, "./model_output_tokenized")
trainer_full = train_model(tokenized_full_papers, model_name, "./model_output_full")

/bin/bash: -c: line 1: unexpected EOF while looking for matching `''
/bin/bash: -c: line 2: syntax error: unexpected end of file


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/tmp/ipykernel_3131/2858322681.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


<pad> 1
</s> 2


Epoch,Training Loss,Validation Loss
1,2.125300,1.909403
2,1.815900,1.748993
3,1.629200,1.736138


...

Epoch,Training Loss,Validation Loss
1,2.076400,2.012161
2,1.783500,1.835283
3,1.479400,1.811634


/opt/conda/lib/python3.11/site-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/tmp/ipykernel_3131/2858322681.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


<pad> 1
</s> 2


Epoch,Training Loss,Validation Loss
1,2.082500,1.804906
2,1.719500,1.641476
3,1.519500,1.624296


/opt/conda/lib/python3.11/site-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
